In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Define key parameters
batch_size = 64
d_model = 512
nhead = 8
num_decoder_layers = 8
dim_feedforward = 2048
dropout = 0.1
vocab_size = 30000
max_len = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=max_len):
        super().__init__()
        self.d_model = d_model
        self.pos_emb = nn.Embedding(max_len, d_model)

        # Define positions
        positions = torch.arange(0, max_len).unsqueeze(0)
        self.register_buffer('positions', positions)
    
    def forward(self, x):
        # Get positions
        pos = self.pos_emb(self.positions)

        # Add positional encoding to input
        return x + pos
    
pos_emb = PositionalEncoding(d_model).to(device)
x = torch.randn(batch_size, max_len, d_model).to(device)
x = pos_emb(x)
print(x.shape)

torch.Size([64, 128, 512])


In [2]:
# Define single Attention Head
class AttentionHead(nn.Module):
    def __init__(self, head_size, masking=True):
        super(AttentionHead, self).__init__()
        self.Q = nn.Linear(d_model, head_size) # Query: What the token wants
        self.K = nn.Linear(d_model, head_size) # Key: What the token has
        self.V = nn.Linear(d_model, head_size) # Value: What the token can communicate if other tokens are interested in it
        self.d_model = d_model
        self.softmax = nn.Softmax(dim=-1)
        self.masking = masking
        self.dropout = nn.Dropout(dropout)
        self.head_size = head_size

    def forward(self, x, padding_mask=None): 
        B, T, C = x.shape
        q = self.Q(x)
        k = self.K(x)
        v = self.V(x)

        attention_weights = q @ k.transpose(-2, -1) / math.sqrt(self.head_size)

        if padding_mask is not None:
            attention_weights = attention_weights.masked_fill(padding_mask.unsqueeze(1), float('-inf'))

        if self.masking: # Causal masking for decoder self-attention
            tril = torch.tril(torch.ones(T, T)).to(device)
            attention_weights = attention_weights.masked_fill(tril == 0, float('-inf'))

        attention_weights = self.softmax(attention_weights)
        attention_weights = self.dropout(attention_weights)
        return attention_weights @ v

x_test = torch.randn(batch_size, max_len, d_model).to(device)
attention_head = AttentionHead(d_model // nhead).to(device)
print(attention_head(x_test).shape)

# Define Multi-Head Attention
class MultiHeadAttention(nn.Module):
    '''Tokens look at each other via Multi-Head Attention'''
    def __init__(self, num_heads, masking=True):
        super(MultiHeadAttention, self).__init__()
        self.head_size = d_model // num_heads
        assert self.head_size * num_heads == d_model, "d_model must be divisible by num_heads"
        self.heads = nn.ModuleList([
            AttentionHead(self.head_size, masking) for _ in range(num_heads)
        ])
        self.proj = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, padding_mask=None):
        out = torch.cat([head(x, padding_mask) for head in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

x_test = torch.randn(batch_size, max_len, d_model).to(device)
multihead_attention = MultiHeadAttention(nhead).to(device)
print(multihead_attention(x_test).shape)

# Define Feedforward Layer
class FeedForward(nn.Module):
    '''Tokens communicate/talk with each other via FeedForward Layer'''
    def __init__(self, d_model):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = self.gelu(self.linear1(x))
        x = self.dropout(self.linear2(x))
        return x

x_test = torch.randn(batch_size, max_len, d_model).to(device)
feedforward = FeedForward(d_model).to(device)
print(feedforward(x_test).shape)
    
# Define Decoder Layer
class Decoder(nn.Module):
    '''MHA(self,masked) + x -> LN -> FF + x -> LN'''
    def __init__(self, d_model, nhead, dropout):
        super(Decoder, self).__init__()
        self.self_attn = MultiHeadAttention(nhead)
        self.ff = FeedForward(d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, decoder_padding_mask=None):
        self_attn_out = self.dropout(self.self_attn(x, decoder_padding_mask))
        x = self.norm1(x + self_attn_out)
        ff_out = self.dropout(self.ff(x))
        x = self.norm2(x + ff_out)
        return x

x_test = torch.randn(batch_size, max_len, d_model).to(device)
decoder = Decoder(d_model, nhead, dropout).to(device)
print(decoder(x_test).shape)
    
# Define GPT-1 model
class GPT_1(nn.Module):
    '''GPT-1 model'''
    def __init__(self, vocab_size, d_model, nhead, num_decoder_layers, dropout, max_len=max_len):
        super(GPT_1, self).__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model, max_len)
        self.decoder_layers = nn.ModuleList([
            Decoder(d_model, nhead, dropout) for _ in range(num_decoder_layers)
        ])
        self.fc = nn.Linear(d_model, vocab_size)
    
    def forward(self, x, decoder_padding_mask=None):
        x = self.token_emb(x)
        x = self.pos_emb(x)
        for layer in self.decoder_layers:
            x = layer(x, decoder_padding_mask)
        x = self.fc(x)
        return x

x_test = torch.randint(0, vocab_size, (batch_size, max_len)).to(device)
gpt_1 = GPT_1(vocab_size, d_model, nhead, num_decoder_layers, dropout).to(device)
print(gpt_1(x_test).shape)

torch.Size([64, 128, 64])
torch.Size([64, 128, 512])
torch.Size([64, 128, 512])
torch.Size([64, 128, 512])
torch.Size([64, 128, 30000])


In [4]:
'''For testing out the trained GPT-1 model'''

from GPT_Decoder import *
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file('bpe.json')
gpt_1 = GPT_1(vocab_size, d_model, nhead, num_decoder_layers, dropout, max_len).to(device)
gpt_1.load_state_dict(torch.load('model/seq_128_d_512_n_8/gpt_1_10.pt'))
test_out = gpt_1.generate()
print(tokenizer.decode(test_out.tolist()))


/tmp/ipykernel_6258/1157239935.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  gpt_1.load_state_dict(torch.load('model/seq_128_d_512_n_8/gpt_1_10.pt'))


The Speciment , released in The History , Sexquour and St. Brey 's , and St. Popen , also being published in or Slimpome of the type rein American SM edition retorines ; @-@ Sand chinGB The Book .
